In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
import ast
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def SQL_push(sql_table_name,python_table,con,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursorUAT.execute(sql_query)
            connUAT.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            con.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\kaggle_data'

In [2]:
sql_check = '''
                select distinct *
                from staging.stg_tmdb_json_movie_metadata stjmm 
                where request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
                order by id
            '''
movie_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
movie_json.sort_values(by='id',inplace=True)
movie_json

,id,request_json
0,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
1,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
2,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
3,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
4,5,"{""adult"": false, ""backdrop_path"": ""/f2t4JbUvQI..."
...,...,...
85459,1144338,"{""adult"": false, ""backdrop_path"": null, ""belon..."
85460,1145358,"{""adult"": false, ""backdrop_path"": ""/4OIJmgez12..."
85461,1146246,"{""adult"": false, ""backdrop_path"": ""/t86SkVd35W..."
85462,1146249,"{""adult"": false, ""backdrop_path"": ""/eNopYAzvtf..."


In [3]:
data_dict_list = []

for i in tqdm(set(movie_json['id'])):
    data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
    data = f'''{data_str}'''
    data_dict = json.loads(data)
    data_dict_list.append(data_dict)
    
movie_metadata = pd.DataFrame(data_dict_list)
movie_metadata

100%|██████████| 85304/85304 [00:21<00:00, 3953.40it/s]


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/9uW5X9fdtfJp0bSW60Eqh0lqcEt.jpg,None,0,"[{'id': 99, 'name': 'Documentary'}]",https://www.netflix.com/title/81074663,524288,tt6939026,[IN],hi,...,2018-04-05,0,26,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,Period. End of Sentence.,False,7.800,238
1,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,2,tt0094675,[FI],fi,...,1988-10-21,0,73,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,,Ariel,False,7.111,346
2,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,3,tt0092149,[FI],fi,...,1986-10-17,0,74,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,,Shadows in Paradise,False,7.295,410
3,False,/f2t4JbUvQIjUF5FstG1zZFAp02N.jpg,None,4000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",https://www.miramax.com/movie/four-rooms/,5,tt0113101,[US],en,...,1995-12-09,4257354,98,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.900,2683
4,False,/iUUpKunmBN5l8goObADBaFHnxQ8.jpg,None,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",,6,tt0107286,[US],en,...,1993-10-15,12136938,109,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.458,349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85299,False,/d0LvPveoC0Tu2CrvoexOgFcp38F.jpg,None,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",None,262120,tt0255713,[IN],hi,...,2001-01-19,0,150,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,None,Zubeidaa,False,5.200,14
85300,False,/2Pbjc94lE67VoD9JJluQJAtQ5Ni.jpg,None,0,"[{'id': 80, 'name': 'Crime'}, {'id': 12, 'name...",None,262123,tt0032533,[US],en,...,1940-12-20,0,62,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,TOUGH HIDES...and STOUT HEARTS!...THE DEAD END...,Give Us Wings,False,6.000,2
85301,False,/uQ9ZK4mo6ZWbNjVOc95lSzKWHFx.jpg,None,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",None,786415,tt11348164,[US],en,...,2022-10-11,0,88,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,Iké Boys,False,5.800,9
85302,False,None,None,0,"[{'id': 10749, 'name': 'Romance'}]",None,524275,tt3715208,[US],en,...,2015-09-03,0,95,[],Released,None,Love in Tokyo,False,6.000,2


In [4]:
# filename = 'movies_metadata' #  index = 35587
# md = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# md = md.drop([19730, 29503, 35587])
# md

In [5]:
movie_metadata_short = movie_metadata[['id','imdb_id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
movie_metadata_short.head(5)

,id,imdb_id,title,original_title,original_language,release_date,status,overview,tagline,adult,popularity,homepage,poster_path,runtime,budget,revenue,vote_average,vote_count
0,524288,tt6939026,Period. End of Sentence.,Period. End of Sentence.,hi,2018-04-05,Released,"In an effort to improve feminine hygiene, a ma...",None,False,0.6451,https://www.netflix.com/title/81074663,/dsCeBj8oabzsHQOGGLPrmrqIvDs.jpg,26,0,0,7.800,238
1,2,tt0094675,Ariel,Ariel,fi,1988-10-21,Released,A Finnish man goes to the city to find a job a...,,False,1.5152,,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,73,0,0,7.111,346
2,3,tt0092149,Shadows in Paradise,Varjoja paratiisissa,fi,1986-10-17,Released,"Nikander, a rubbish collector and would-be ent...",,False,1.2309,,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,74,0,0,7.295,410
3,5,tt0113101,Four Rooms,Four Rooms,en,1995-12-09,Released,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,False,3.1509,https://www.miramax.com/movie/four-rooms/,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,98,4000000,4257354,5.900,2683
4,6,tt0107286,Judgment Night,Judgment Night,en,1993-10-15,Released,"Four young friends, while taking a shortcut en...",Don't move. Don't whisper. Don't even breathe.,False,1.7698,,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,109,21000000,12136938,6.458,349


In [6]:
movie_metadata[['id','production_companies']]

,id,production_companies
0,524288,"[{'id': 22486, 'logo_path': '/a101r8HZ9dk6xvXy..."
1,2,"[{'id': 2303, 'logo_path': None, 'name': 'Vill..."
2,3,"[{'id': 2303, 'logo_path': None, 'name': 'Vill..."
3,5,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv..."
4,6,"[{'id': 1644, 'logo_path': '/l5JFudbLN3vr1sSkj..."
...,...,...
85299,262120,"[{'id': 91496, 'logo_path': None, 'name': 'Shy..."
85300,262123,"[{'id': 33, 'logo_path': '/3wwjVpkZtnog6lSKzWD..."
85301,786415,[]
85302,524275,[]


In [7]:
movie_metadata[['id','production_companies']]['production_companies'][4]

[{'id': 1644,
  'logo_path': '/l5JFudbLN3vr1sSkjRoqF7To8Ky.png',
  'name': 'Largo Entertainment',
  'origin_country': 'US'},
 {'id': 182,
  'logo_path': '/1BPbXmwi1DZS1Qp0koK7AuponY6.png',
  'name': 'JVC',
  'origin_country': 'JP'},
 {'id': 33,
  'logo_path': '/3wwjVpkZtnog6lSKzWDjvw2Yi00.png',
  'name': 'Universal Pictures',
  'origin_country': 'US'}]

In [8]:
movie_metadata[['id','production_countries']]

,id,production_countries
0,524288,"[{'iso_3166_1': 'IN', 'name': 'India'}]"
1,2,"[{'iso_3166_1': 'FI', 'name': 'Finland'}]"
2,3,"[{'iso_3166_1': 'FI', 'name': 'Finland'}]"
3,5,"[{'iso_3166_1': 'US', 'name': 'United States o..."
4,6,"[{'iso_3166_1': 'US', 'name': 'United States o..."
...,...,...
85299,262120,"[{'iso_3166_1': 'IN', 'name': 'India'}]"
85300,262123,"[{'iso_3166_1': 'US', 'name': 'United States o..."
85301,786415,[]
85302,524275,[]


In [9]:
belongs_to_collection_raw = movie_metadata[['id','belongs_to_collection']]
belongs_to_collection_raw.replace([None],[np.nan],inplace=True)
belongs_to_collection_raw = belongs_to_collection_raw[belongs_to_collection_raw['belongs_to_collection'].isnull()==False].reset_index(drop=True)
belongs_to_collection_raw.rename(columns={'id':'tmdbid'},inplace=True)

belongs_to_collection_raw

,tmdbid,belongs_to_collection
0,2,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol..."
1,3,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol..."
2,11,"{'id': 10, 'name': 'Star Wars Collection', 'po..."
3,12,"{'id': 137697, 'name': 'Finding Nemo Collectio..."
4,21,"{'id': 252313, 'name': 'The Endless Summer Col..."
...,...,...
9175,524047,"{'id': 840465, 'name': 'Greenland Collection',..."
9176,786345,"{'id': 1020749, 'name': 'Viduthalai Collection..."
9177,786357,"{'id': 881669, 'name': 'You vs. Wild (Collecti..."
9178,524222,"{'id': 1413547, 'name': 'Bəxt üzüyü', 'poster_..."


In [10]:
dict_data = []

for i in tqdm(set(belongs_to_collection_raw['tmdbid'])):
    datadf = belongs_to_collection_raw[belongs_to_collection_raw['tmdbid']==i]['belongs_to_collection'].values[0]
    datadf['collection_id'] = datadf.pop('id')
    datadf.update({'id':i})
    dict_data.append(datadf)

df = pd.DataFrame(dict_data)

df

100%|██████████| 9180/9180 [00:01<00:00, 4855.95it/s]


,name,poster_path,backdrop_path,collection_id,id
0,Aki Kaurismäki's Proletariat Trilogy,/bUrReoZFLGti6ehkBW0xw8f12MT.jpg,/zAUItK1Nr473DIe8gWMsZ0DMR7L.jpg,1382526,2
1,Aki Kaurismäki's Proletariat Trilogy,/bUrReoZFLGti6ehkBW0xw8f12MT.jpg,/zAUItK1Nr473DIe8gWMsZ0DMR7L.jpg,1382526,3
2,Star Wars Collection,/aSrMJYmQX8kpF26LijkCsYhBMvm.jpg,/zZDkgOmFMVYpGAkR9Tkxw0CRnxX.jpg,10,11
3,Finding Nemo Collection,/cCovtlN16ykvyFYnzKyv3dFtceG.jpg,/bcWmv9xsKv6xXmdz5OP2wefHcdx.jpg,137697,12
4,The Endless Summer Collection,/z6HnwsOtTUilNMcAbgEkmqUH9S5.jpg,/rbeqwDmy0yPMxBqA2ELvu08okcn.jpg,252313,21
...,...,...,...,...,...
9175,Pitbull Collection,/4SB9F6P9K9445EJZoYfNZ8a56RL.jpg,/xRwCYxTrw9B7ToH98IYHglAVQxw.jpg,402380,425961
9176,Single Santa Seeks Mrs. Claus Collection,None,None,662550,393196
9177,Pyaar Ka Punchnama Collection,/qGE8SUFczhqpXUUVSb5i0RXC5ca.jpg,/tCcXxujtGoSjbxrNmQkLLMiEBim.jpg,505479,65521
9178,They Call Me Bruce Collection,/xjVXgVDzgF7I1zm0AymBRbPWRn2.jpg,None,315750,65522


In [16]:
df.groupby('id').agg(idcount=('id','count')).reset_index().sort_values(by=['idcount','id'])

,id,idcount
0,2,1
1,3,1
2,11,1
3,12,1
4,21,1
...,...,...
9175,1094844,1
9176,1115710,1
9177,1124663,1
9178,1136736,1


In [13]:
df.sort_values(by='id',ascending=True)

,name,poster_path,backdrop_path,collection_id,id
0,Aki Kaurismäki's Proletariat Trilogy,/bUrReoZFLGti6ehkBW0xw8f12MT.jpg,/zAUItK1Nr473DIe8gWMsZ0DMR7L.jpg,1382526,2
1,Aki Kaurismäki's Proletariat Trilogy,/bUrReoZFLGti6ehkBW0xw8f12MT.jpg,/zAUItK1Nr473DIe8gWMsZ0DMR7L.jpg,1382526,3
2,Star Wars Collection,/aSrMJYmQX8kpF26LijkCsYhBMvm.jpg,/zZDkgOmFMVYpGAkR9Tkxw0CRnxX.jpg,10,11
3,Finding Nemo Collection,/cCovtlN16ykvyFYnzKyv3dFtceG.jpg,/bcWmv9xsKv6xXmdz5OP2wefHcdx.jpg,137697,12
4,The Endless Summer Collection,/z6HnwsOtTUilNMcAbgEkmqUH9S5.jpg,/rbeqwDmy0yPMxBqA2ELvu08okcn.jpg,252313,21
...,...,...,...,...,...
4436,Ape vs. Collection,/zLqSnZUYW2y333PaSrR45fS1T0V.jpg,None,1102610,1094844
606,The Mount Collection,None,None,1115715,1115710
3402,Patterns of Evidence Collection,/kFIe8e2LbuPpIg502ZKmhPtHO6e.jpg,/rMeOaZYQvwLJOusyDBDN6keDkyU.jpg,719840,1124663
6873,Lust Stories Collection,/p147Y0AMgJruqSY6Ct05yytxDxd.jpg,/aTCmI9SnQF6Q4SWPTaLTIKIH9Um.jpg,1136734,1136736


In [21]:
genres_raw = md[['id','genres']]
# genres = genres_raw['genres'][0]
# genres
genres_raw

,id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...
45461,439050,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,111109,"[{'id': 18, 'name': 'Drama'}]"
45463,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,[]


In [51]:
data_dict = ast.literal_eval(genres_raw['genres'][0])
df = pd.DataFrame(data_dict)
df

,id,name
0,16,Animation
1,35,Comedy
2,10751,Family


In [22]:
spoken_languages_raw = md[['id','spoken_languages']]
# genres = genres_raw['genres'][0]
# genres
spoken_languages_raw

,id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,"[{'iso_639_1': 'en', 'name': 'English'}]"
...,...,...
45461,439050,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]"
45462,111109,"[{'iso_639_1': 'tl', 'name': ''}]"
45463,67758,"[{'iso_639_1': 'en', 'name': 'English'}]"
45464,227506,[]


In [ ]:
filename = 'credits'
credits = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
credits

In [ ]:
credits_cast = credits[['id','cast']]
credits_cast

In [ ]:
ast.literal_eval(credits[credits['id']==862]['crew'].values[0])

In [14]:
df_cast = pd.DataFrame(ast.literal_eval(credits[credits['id']==862]['cast'].values[0]))
df_cast


,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
5,19,Hamm (voice),52fe4284c3a36847f8024fa9,2,7907,John Ratzenberger,5,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg
6,20,Bo Peep (voice),52fe4284c3a36847f8024fad,1,8873,Annie Potts,6,/eryXT84RL41jHSJcMy4kS3u9y6w.jpg
7,26,Andy (voice),52fe4284c3a36847f8024fc1,0,1116442,John Morris,7,/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg
8,22,Sid (voice),52fe4284c3a36847f8024fb1,2,12901,Erik von Detten,8,/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg
9,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,1,12133,Laurie Metcalf,9,/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg


In [10]:
# data_str = smd[smd.id==863]['keywords'].values[0]
# a = ast.literal_eval(data_str)
# a[0]['name']

In [2]:
filename = 'keywords'
keywords = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [9]:
ast.literal_eval(keywords[keywords['id']==11]['keywords'].values[0])

[{'id': 803, 'name': 'android'},
 {'id': 4270, 'name': 'galaxy'},
 {'id': 4271, 'name': 'hermit'},
 {'id': 10013, 'name': 'death star'},
 {'id': 10016, 'name': 'lightsaber'},
 {'id': 10527, 'name': 'jedi'},
 {'id': 11107, 'name': 'rescue mission'},
 {'id': 11195, 'name': 'empire'},
 {'id': 11196, 'name': 'rebellion'},
 {'id': 13194, 'name': 'planet'},
 {'id': 155336, 'name': 'smuggler'},
 {'id': 160134, 'name': 'the force'},
 {'id': 161176, 'name': 'space opera'},
 {'id': 163295, 'name': 'galactic war'},
 {'id': 178710, 'name': 'stormtrooper'},
 {'id': 178712, 'name': 'totalitarianism'}]

In [8]:
ast.literal_eval(keywords[keywords['id']==11]['keywords'].values[0])
data_json = keywords[keywords['id']==11]['keywords'].values[0]
data_json
# json.loads(data_json.values[0])

"[{'id': 803, 'name': 'android'}, {'id': 4270, 'name': 'galaxy'}, {'id': 4271, 'name': 'hermit'}, {'id': 10013, 'name': 'death star'}, {'id': 10016, 'name': 'lightsaber'}, {'id': 10527, 'name': 'jedi'}, {'id': 11107, 'name': 'rescue mission'}, {'id': 11195, 'name': 'empire'}, {'id': 11196, 'name': 'rebellion'}, {'id': 13194, 'name': 'planet'}, {'id': 155336, 'name': 'smuggler'}, {'id': 160134, 'name': 'the force'}, {'id': 161176, 'name': 'space opera'}, {'id': 163295, 'name': 'galactic war'}, {'id': 178710, 'name': 'stormtrooper'}, {'id': 178712, 'name': 'totalitarianism'}]"

In [7]:
# filename = 'ratings'
# ratings = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# ratings

In [8]:
# filename = 'links'
# links = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# links

In [9]:
# filename = 'links_small'
# links_small = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# links_small